In [52]:
from transformers import AutoTokenizer, CamembertForQuestionAnswering,pipeline
import torch

In [53]:
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = CamembertForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

You are using a model of type roberta to instantiate a model of type camembert. This is not supported for all configurations of models and can yield errors.


In [54]:
import json

with open('data2.json', 'r') as f:
    data = json.load(f)

In [55]:
type(data)

list

In [56]:
# Define the corpus of documents
corpus=[]
for i in range(len(data)):
    if data[i]['context'] not in ['\n\t',None]:
        corpus.append(data[i]['context']) 
    if data[i]['context2'] not in ['\n\t',None]:
        corpus.append(data[i]['context2'])

In [57]:
len(corpus)

267

In [58]:
pip install stop_words

Note: you may need to restart the kernel to use updated packages.


In [59]:
from stop_words import get_stop_words

french_stop_words = get_stop_words('french')


In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define the question
question2 = "Qulle est la date de création de laboratoire de Métrologie de l’Armée de l’Air «PMEL» ?"
question='Les  différents travaux en matière de métrologie de «PMEL» lui ont permis d’être reconnu par qui?'
question3='Les  différents travaux en matière de métrologie de «PMEL» lui ont permis d’être reconnu pour quels domaines?'
# Preprocess the corpus
def TF_IDF(question):
    vectorizer = TfidfVectorizer(stop_words= french_stop_words)
    corpus_tfidf = vectorizer.fit_transform(corpus)

    # Compute the TF-IDF scores for the question
    question_tfidf = vectorizer.transform([question])

    # Compute the cosine similarity between the question and each document in the corpus
    cos_similarities = cosine_similarity(question_tfidf, corpus_tfidf)

    # Select the document with the highest cosine similarity as the best context for the question
    best_doc_idx = cos_similarities.argmax()
    best_doc = corpus[best_doc_idx]

    print("Best context for question '{}': '{}'".format(question, best_doc))

Best context for question 'Les  différents travaux en matière de métrologie de «PMEL» lui ont permis d’être reconnu par qui?': 'Le laboratoire de Métrologie de l’Armée de l’Air «PMEL» a été créé en Novembre 1985. Ses différents travaux en matière de métrologie lui ont permis d’être reconnu par le Centre de Métrologie Américain (AFMETCAL) depuis 1989 pour les domaines Electrique, Physique et Dimensionnel et d’être accrédité par le Comité Français d’Accréditation (COFRAC) depuis l’année 2000 pour le domaine « Electricité–Magnétisme ».'


In [82]:
best_doc

'Le laboratoire de Métrologie de l’Armée de l’Air «PMEL» a été créé en Novembre 1985. Ses différents travaux en matière de métrologie lui ont permis d’être reconnu par le Centre de Métrologie Américain (AFMETCAL) depuis 1989 pour les domaines Electrique, Physique et Dimensionnel et d’être accrédité par le Comité Français d’Accréditation (COFRAC) depuis l’année 2000 pour le domaine « Electricité–Magnétisme ».'

In [83]:
nlp = pipeline("question-answering", model="deepset/roberta-base-squad2", tokenizer="deepset/roberta-base-squad2")

inputs = tokenizer(question,best_doc, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
result = nlp(question=question, context=best_doc)


print("\nRéponse: ",result)


Réponse:  {'score': 0.1797487586736679, 'start': 170, 'end': 200, 'answer': 'Centre de Métrologie Américain'}
